In [1]:
#Importing the necessary libraries
from osgeo import gdal
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd

In [2]:
import xarray as xr
import glob

In [3]:
pwd


'/Users/dawn.urycki/Repos/temp-data'

In [4]:
tempfiles = glob.glob(r'DATA/nc_files/*.nc')
#tempfiles

In [5]:
# Check
tempdata1 = Dataset(tempfiles[5])
print (tempdata1)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Predicted future stream temperatures
    institution: https://www.fisheries.noaa.gov/region/west-coast/northwest-science
    source: URL TBD
    references: Fullerton et al. (2024). NOAA Data Report
    history: A. H. Fullerton Sun Oct 27 09:27:25 2024
    Conventions: CF-1.0
    dimensions(sizes): date(54787), COMID(36), GCM(10)
    variables(dimensions): float64 date(date), int32 COMID(COMID), int32 GCM(GCM), float64 prd.stream_temp(GCM, COMID, date)
    groups: 


In [6]:
preds = xr.open_dataset(tempfiles[0])

In [7]:
preds

<xarray.Dataset> Size: 185MB
Dimensions:          (date: 54787, COMID: 42, GCM: 10)
Coordinates:
  * date             (date) datetime64[ns] 438kB 1950-01-01 ... 2099-12-31
  * COMID            (COMID) int32 168B 23066771 23066773 ... 24121964 24121984
  * GCM              (GCM) int32 40B 1 2 3 4 5 6 7 8 9 10
Data variables:
    prd.stream_temp  (GCM, COMID, date) float64 184MB ...
Attributes:
    title:        Predicted future stream temperatures
    institution:  https://www.fisheries.noaa.gov/region/west-coast/northwest-...
    source:       URL TBD
    references:   Fullerton et al. (2024). NOAA Data Report
    history:      A. H. Fullerton Sat Oct 26 18:25:26 2024
    Conventions:  CF-1.0

In [15]:
# import py7zr
# with py7zr.SevenZipFile('./DATA/NHDPlusV21_PN_17_NHDSnapshot_08.7z', mode='r') as z:
#    z.extractall()

In [8]:
nhd_data = glob.glob('./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/*')
nhd_data

['./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDFCode.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDReachCrossReference.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDReachCode_Comid.dbf']

In [9]:
dbfhydro = glob.glob(r'./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/*/*.dbf') 
dbfhydro

['./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDFlowline.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDLineEventFC.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDArea.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDPoint.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDAreaEventFC.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDPointEventFC.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDLine.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/Hydrography/NHDWaterbody.dbf']

In [10]:
dbfref = glob.glob(r'./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/*.dbf') 
dbfref

['./DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDFCode.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDReachCrossReference.dbf',
 './DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/NHDReachCode_Comid.dbf']

In [11]:
reach_comid = gpd.read_file(dbfref[2])
reach_comid

,COMID,REACHCODE,REACHSMDAT,RESOLUTION,GNIS_ID,GNIS_NAME
0,0,17060306010961,2012-06-01,None,None,None
1,0,17050110000038,2012-06-01,None,None,None
2,0,17080003006393,2012-06-01,None,None,None
3,0,17010104125860,2012-06-01,None,None,None
4,0,17060306011186,2012-06-01,None,None,None
...,...,...,...,...,...,...
254902,918470590,17050123135071,2012-06-01,Medium,None,None
254903,919670591,17120007003351,2012-06-01,Medium,None,None
254904,919200592,17100106060628,2012-06-01,Medium,None,None
254905,919640593,17120004006262,2012-06-01,Medium,None,None


In [12]:
flowlines = gpd.read_file(dbfhydro[0])
flowlines

/opt/miniconda3/envs/temp-data-env/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(


,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
0,9301535,1999-10-08,Medium,None,None,0.999,09040002011583,Uninitialized,0,StreamRiver,46006,0.011661,True,0,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
1,22968128,2001-03-02,Medium,None,None,0.134,10030104009812,Uninitialized,0,StreamRiver,46003,0.001671,True,0,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
2,22968138,2001-03-02,Medium,None,None,0.217,10030104009813,Uninitialized,0,StreamRiver,46003,0.002145,True,0,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
3,23135995,2001-02-07,Medium,None,None,0.916,16010102003533,Uninitialized,0,StreamRiver,46003,0.010671,True,0,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
4,23135997,2001-02-07,Medium,None,None,1.865,16010102003534,Uninitialized,0,StreamRiver,46003,0.019193,True,0,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272335,24545039,2005-08-27,Medium,None,None,2.847,17120009001542,Uninitialized,0,StreamRiver,46003,0.029718,True,0,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
272336,24545041,2005-08-27,Medium,None,None,2.005,17120009001543,Uninitialized,0,StreamRiver,46003,0.020023,True,0,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
272337,23706720,2001-02-28,Medium,None,None,4.217,18010201002017,Uninitialized,0,StreamRiver,46003,0.043621,True,0,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."
272338,23931942,2001-03-14,Medium,1127294,South Fork Keene Creek,0.029,18010206012384,Uninitialized,0,StreamRiver,46006,0.000284,True,0,"LINESTRING Z (-122.47671 42.08355 0, -122.4765..."


In [23]:
#flowlines[:50].explore()

In [13]:
flowlines.FTYPE.unique()

array(['StreamRiver', 'ArtificialPath', 'CanalDitch', 'Pipeline',
       'Connector', 'Coastline'], dtype=object)

In [14]:
predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])

In [15]:
150*365


54750

In [36]:
predsdf

prd.stream_temp
COMID    GCM date                       
23066771 1   1950-01-01              NaN
             1950-01-02              NaN
             1950-01-03              NaN
             1950-01-04              NaN
             1950-01-05              NaN
...                                  ...
24121984 10  2099-12-27         9.303043
             2099-12-28         8.525719
             2099-12-29         7.030612
             2099-12-30         8.553759
             2099-12-31         7.153554

[23010540 rows x 1 columns]

In [16]:
cur_comids = predsdf.index.get_level_values('COMID').unique()

In [17]:
len(predsdf.index.get_level_values('COMID').unique())

42

In [18]:
gcm1 = predsdf.xs(1, level = 1)

In [19]:
gcm1

prd.stream_temp
COMID    date                       
23066771 1950-01-01              NaN
         1950-01-02              NaN
         1950-01-03              NaN
         1950-01-04              NaN
         1950-01-05              NaN
...                              ...
24121984 2099-12-27         5.770917
         2099-12-28         6.822931
         2099-12-29         7.623787
         2099-12-30         8.221618
         2099-12-31         7.753777

[2301054 rows x 1 columns]

In [41]:
import pandas as pd

In [20]:
geometries = flowlines[['COMID', 'geometry']].set_index('COMID')
geometries


,geometry
COMID,
9301535,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...
24545039,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
24545041,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
23706720,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."


In [43]:
gcm1.index.get_level_values(0)

Index([23066771, 23066771, 23066771, 23066771, 23066771, 23066771, 23066771,
       23066771, 23066771, 23066771,
       ...
       24121984, 24121984, 24121984, 24121984, 24121984, 24121984, 24121984,
       24121984, 24121984, 24121984],
      dtype='int32', name='COMID', length=2301054)

In [21]:
gcm1_flat = gcm1.reset_index()

In [22]:
gcm1_flat


,COMID,date,prd.stream_temp
0,23066771,1950-01-01,NaN
1,23066771,1950-01-02,NaN
2,23066771,1950-01-03,NaN
3,23066771,1950-01-04,NaN
4,23066771,1950-01-05,NaN
...,...,...,...
2301049,24121984,2099-12-27,5.770917
2301050,24121984,2099-12-28,6.822931
2301051,24121984,2099-12-29,7.623787
2301052,24121984,2099-12-30,8.221618


In [23]:
gcm1_flat.COMID.unique()

array([23066771, 23066773, 23066775, 23067095, 23067097, 23067181,
       24120666, 24120668, 24120670, 24120672, 24120674, 24120676,
       24120678, 24120680, 24120682, 24120684, 24120686, 24120688,
       24120690, 24120692, 24120694, 24120696, 24120698, 24120700,
       24120704, 24120706, 24120708, 24120710, 24120712, 24120714,
       24120716, 24120718, 24120738, 24121550, 24121932, 24121936,
       24121938, 24121950, 24121952, 24121954, 24121964, 24121984],
      dtype=int32)

In [24]:
geometries.index.unique()

Index([ 9301535, 22968128, 22968138, 23135995, 23135997, 23135999, 23136001,
       23136003, 23136005, 23136007,
       ...
       24545029, 24545031, 24545033, 24545035, 24545037, 24545039, 24545041,
       23706720, 23931942, 24279568],
      dtype='int32', name='COMID', length=272340)

In [48]:
%%time
gcm1_flat['geometries'] = gcm1_flat.COMID
gcm1_flat

CPU times: user 5.11 ms, sys: 4.23 ms, total: 9.34 ms
Wall time: 6 ms


,COMID,date,prd.stream_temp,geometries
0,23066771,1950-01-01,NaN,23066771
1,23066771,1950-01-02,NaN,23066771
2,23066771,1950-01-03,NaN,23066771
3,23066771,1950-01-04,NaN,23066771
4,23066771,1950-01-05,NaN,23066771
...,...,...,...,...
2301049,24121984,2099-12-27,5.770917,24121984
2301050,24121984,2099-12-28,6.822931,24121984
2301051,24121984,2099-12-29,7.623787,24121984
2301052,24121984,2099-12-30,8.221618,24121984


In [49]:
geomdict = geometries.to_dict()

In [50]:
for cid in gcm1_flat.geometries.unique():
    print(geomdict[str(cid)])

KeyError: '23066771'

In [51]:
set2 = set(gcm1_flat.COMID.unique())

In [52]:
gcm1_flat.COMID.unique().dtype

dtype('int32')

In [53]:
set1 = set(geometries.index.unique())

In [54]:
len(set2)

42

In [55]:
len(set1.intersection(set2))

42

In [56]:
geometries.index.unique().dtype

dtype('int32')

In [57]:
a = gcm1_flat.geometries[0]

In [58]:
geomdict[str(a)]

KeyError: '23066771'

In [59]:
list(geomdict.keys())

['geometry']

In [60]:
geometries

,geometry
COMID,
9301535,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...
24545039,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
24545041,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
23706720,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."


In [100]:
geometries.columns.values

array(['geometry'], dtype=object)

In [101]:
geometries.loc[a].values

<GeometryArray>
[<LINESTRING Z (-117.492 47.781 0, -117.492 47.781 0, -117.492 47.78 0, -117....>]
Length: 1, dtype: geometry

In [107]:
geometries.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 272340 entries, 9301535 to 24279568
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  272340 non-null  geometry
dtypes: geometry(1)
memory usage: 9.2 MB


In [105]:
gcm1_flat['COMID'].map(geometries)

ValueError: 2

In [113]:
geo_gcm1 = gpd.GeoDataFrame(gcm1_flat)

In [119]:
trim_geos = geometries.loc[list(set2.intersection(set1))]

In [128]:
trim_geos

,geometry
COMID,
23026816,"LINESTRING Z (-117.15635 47.96456 0, -117.1584..."
23026818,"LINESTRING Z (-117.14009 47.93361 0, -117.1400..."
23026820,"LINESTRING Z (-117.33586 47.92632 0, -117.3361..."
23026822,"LINESTRING Z (-117.33379 47.95037 0, -117.3337..."
23026824,"LINESTRING Z (-117.33587 47.95635 0, -117.3347..."
23026826,"LINESTRING Z (-117.33448 47.96387 0, -117.3348..."
23026828,"LINESTRING Z (-117.32955 47.98265 0, -117.3306..."
23026708,"LINESTRING Z (-117.49237 47.78126 0, -117.4922..."
23026710,"LINESTRING Z (-117.48148 47.77784 0, -117.4820..."


In [120]:
trim_geos.to_file('test_output.gpkg', driver = "GPKG")

In [121]:
pwd

'/Users/dawn.urycki/Repos/temp-data'

In [122]:
newdata = gpd.read_file('test_output.gpkg')

In [129]:
newdata

,COMID,geometry
0,23026816,"LINESTRING Z (-117.15635 47.96456 0, -117.1584..."
1,23026818,"LINESTRING Z (-117.14009 47.93361 0, -117.1400..."
2,23026820,"LINESTRING Z (-117.33586 47.92632 0, -117.3361..."
3,23026822,"LINESTRING Z (-117.33379 47.95037 0, -117.3337..."
4,23026824,"LINESTRING Z (-117.33587 47.95635 0, -117.3347..."
5,23026826,"LINESTRING Z (-117.33448 47.96387 0, -117.3348..."
6,23026828,"LINESTRING Z (-117.32955 47.98265 0, -117.3306..."
7,23026708,"LINESTRING Z (-117.49237 47.78126 0, -117.4922..."
8,23026710,"LINESTRING Z (-117.48148 47.77784 0, -117.4820..."
9,23026712,"LINESTRING Z (-117.47747 47.77405 0, -117.4784..."


In [133]:
type(newdata)

geopandas.geodataframe.GeoDataFrame

In [134]:
newdata.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [141]:
newdata.to_file('output2.gpkg', layer = 'polygons', driver = 'GPKG')

In [142]:
gpd.list_layers('output2.gpkg')

,name,geometry_type
0,polygons,LineString Z


In [145]:
gcm1_test = gcm1_flat.copy()

In [146]:
gcm1_test['gcm2'] = 2
gcm1_test

,COMID,date,prd.stream_temp,gcm2
0,23026708,1950-01-01,NaN,2
1,23026708,1950-01-02,NaN,2
2,23026708,1950-01-03,1.440884,2
3,23026708,1950-01-04,0.782451,2
4,23026708,1950-01-05,0.000000,2
...,...,...,...,...
2903706,947010262,2099-12-27,4.693037,2
2903707,947010262,2099-12-28,5.052310,2
2903708,947010262,2099-12-29,5.240782,2
2903709,947010262,2099-12-30,5.344503,2


In [148]:
gcm1_test['gcm3'] = 3


In [157]:
geometries

,geometry
COMID,
9301535,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...
24545039,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
24545041,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
23706720,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."


In [160]:
pd.concat([geometries, gcm1_test.set_index('COMID')], axis = 1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [156]:
gcm1_gpd = gpd.GeoDataFrame(gcm1_test, geometry = geometries)

ValueError: Must pass array with one dimension only.

In [152]:
pd.concat([gcm1_test.set_index('COMID'), geometries], axis = 1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [144]:
gcm1_flat.to_file('output2.gpkg', layer="gcm1", mode="a", driver="GPKG")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [161]:
gcm1_flat

,COMID,date,prd.stream_temp
0,23026708,1950-01-01,NaN
1,23026708,1950-01-02,NaN
2,23026708,1950-01-03,1.440884
3,23026708,1950-01-04,0.782451
4,23026708,1950-01-05,0.000000
...,...,...,...
2903706,947010262,2099-12-27,4.693037
2903707,947010262,2099-12-28,5.052310
2903708,947010262,2099-12-29,5.240782
2903709,947010262,2099-12-30,5.344503


In [197]:
#geomdict

In [176]:
geometries.columns


Index(['geometry'], dtype='object')

In [177]:
gcm1_flat.columns

Index(['COMID', 'date', 'prd.stream_temp'], dtype='object')

In [181]:
geodict = geometries.to_geo_dict()

In [179]:
gcm1_flat


,COMID,date,prd.stream_temp
0,23026708,1950-01-01,NaN
1,23026708,1950-01-02,NaN
2,23026708,1950-01-03,1.440884
3,23026708,1950-01-04,0.782451
4,23026708,1950-01-05,0.000000
...,...,...,...
2903706,947010262,2099-12-27,4.693037
2903707,947010262,2099-12-28,5.052310
2903708,947010262,2099-12-29,5.240782
2903709,947010262,2099-12-30,5.344503


In [190]:
geodict.keys()

dict_keys(['type', 'features'])

### Workflow:
1. Create geopackage from nhd, with comid and geometries as columns.
2. Check. (re-open, list layers)
3. Append each gcm as new layer.

In [195]:
flowlines

,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
0,9301535,1999-10-08,Medium,None,None,0.999,09040002011583,Uninitialized,0,StreamRiver,46006,0.011661,True,0,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
1,22968128,2001-03-02,Medium,None,None,0.134,10030104009812,Uninitialized,0,StreamRiver,46003,0.001671,True,0,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
2,22968138,2001-03-02,Medium,None,None,0.217,10030104009813,Uninitialized,0,StreamRiver,46003,0.002145,True,0,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
3,23135995,2001-02-07,Medium,None,None,0.916,16010102003533,Uninitialized,0,StreamRiver,46003,0.010671,True,0,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
4,23135997,2001-02-07,Medium,None,None,1.865,16010102003534,Uninitialized,0,StreamRiver,46003,0.019193,True,0,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272335,24545039,2005-08-27,Medium,None,None,2.847,17120009001542,Uninitialized,0,StreamRiver,46003,0.029718,True,0,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
272336,24545041,2005-08-27,Medium,None,None,2.005,17120009001543,Uninitialized,0,StreamRiver,46003,0.020023,True,0,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
272337,23706720,2001-02-28,Medium,None,None,4.217,18010201002017,Uninitialized,0,StreamRiver,46003,0.043621,True,0,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."
272338,23931942,2001-03-14,Medium,1127294,South Fork Keene Creek,0.029,18010206012384,Uninitialized,0,StreamRiver,46006,0.000284,True,0,"LINESTRING Z (-122.47671 42.08355 0, -122.4765..."


In [196]:
gcm1_flat

,COMID,date,prd.stream_temp
0,23026708,1950-01-01,NaN
1,23026708,1950-01-02,NaN
2,23026708,1950-01-03,1.440884
3,23026708,1950-01-04,0.782451
4,23026708,1950-01-05,0.000000
...,...,...,...
2903706,947010262,2099-12-27,4.693037
2903707,947010262,2099-12-28,5.052310
2903708,947010262,2099-12-29,5.240782
2903709,947010262,2099-12-30,5.344503


In [199]:
cur_flowlines = flowlines.loc[flowlines.COMID.isin(cur_comids)]

In [200]:
cur_flowlines


,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
39625,23026708,2001-02-27,Medium,1522311,Little Spokane River,6.579,17010308000018,With Digitized,0,StreamRiver,46006,0.074359,True,0,"LINESTRING Z (-117.49237 47.78126 0, -117.4922..."
39626,23026710,2001-02-27,Medium,1522311,Little Spokane River,1.730,17010308000019,With Digitized,0,StreamRiver,46006,0.019356,True,0,"LINESTRING Z (-117.48148 47.77784 0, -117.4820..."
39627,23026712,2001-02-27,Medium,1522311,Little Spokane River,0.552,17010308000020,With Digitized,0,StreamRiver,46006,0.005898,True,0,"LINESTRING Z (-117.47747 47.77405 0, -117.4784..."
39628,23026714,2001-02-27,Medium,1522311,Little Spokane River,4.881,17010308000021,With Digitized,0,StreamRiver,46006,0.056356,True,0,"LINESTRING Z (-117.43736 47.77257 0, -117.4390..."
39629,23026716,2001-02-27,Medium,1522311,Little Spokane River,2.185,17010308000022,With Digitized,0,StreamRiver,46006,0.024544,True,0,"LINESTRING Z (-117.42139 47.78146 0, -117.4217..."
39630,23026718,2001-02-27,Medium,1522311,Little Spokane River,0.542,17010308000023,With Digitized,0,StreamRiver,46006,0.006188,True,0,"LINESTRING Z (-117.41655 47.78288 0, -117.4168..."
39631,23026720,2001-02-27,Medium,1522311,Little Spokane River,3.435,17010308000024,With Digitized,0,StreamRiver,46006,0.040026,True,0,"LINESTRING Z (-117.38456 47.79559 0, -117.3846..."
39632,23026722,2001-02-27,Medium,1504371,Deadman Creek,0.335,17010308000025,With Digitized,0,StreamRiver,46006,0.004426,True,0,"LINESTRING Z (-117.38015 47.79566 0, -117.3819..."
39633,23026724,2001-02-27,Medium,1504371,Deadman Creek,2.626,17010308000026,With Digitized,0,StreamRiver,46006,0.027978,True,0,"LINESTRING Z (-117.36474 47.77768 0, -117.3647..."
39634,23026726,2001-02-27,Medium,1504371,Deadman Creek,2.512,17010308000027,With Digitized,0,StreamRiver,46006,0.030440,True,0,"LINESTRING Z (-117.33766 47.78557 0, -117.3389..."


In [201]:
len(cur_flowlines)

53

In [202]:
cur_flowlines.to_file('outputlines.gpkg', layer = 'flowlines', driver = 'GPKG')

In [205]:
test_lines = gpd.read_file('outputlines.gpkg')

In [209]:
test_lines.explore()

In [210]:
predsdf

prd.stream_temp
COMID     GCM date                       
23026708  1   1950-01-01              NaN
              1950-01-02              NaN
              1950-01-03         1.440884
              1950-01-04         0.782451
              1950-01-05         0.000000
...                                   ...
947010262 10  2099-12-27         6.673878
              2099-12-28         6.390356
              2099-12-29         5.624401
              2099-12-30         7.018028
              2099-12-31         5.834494

[29037110 rows x 1 columns]

In [211]:
gcm1_flat.to_file('outputlines.gpkg', layer="gcm1", mode="a", driver="GPKG")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [213]:
import sqlite3

In [214]:
conn = sqlite3.connect('outputlines.gpkg')

In [215]:
gcm1_flat.to_sql('gcm1', conn, index = False)
conn.close()

In [216]:
gpd.list_layers('outputlines.gpkg')

/opt/miniconda3/envs/geopd-env/lib/python3.12/site-packages/pyogrio/core.py:130: RuntimeWarning: Field format 'TIMESTAMP' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


,name,geometry_type
0,flowlines,LineString Z
1,gcm1,None


In [218]:
gpd.read_file('outputlines.gpkg', layer = 'gcm1')

/opt/miniconda3/envs/geopd-env/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Field format 'TIMESTAMP' not supported
  return ogr_read(


,COMID,date,prd.stream_temp
0,23026708,1950-01-01 00:00:00,NaN
1,23026708,1950-01-02 00:00:00,NaN
2,23026708,1950-01-03 00:00:00,1.440884
3,23026708,1950-01-04 00:00:00,0.782451
4,23026708,1950-01-05 00:00:00,0.000000
...,...,...,...
2903706,947010262,2099-12-27 00:00:00,4.693037
2903707,947010262,2099-12-28 00:00:00,5.052310
2903708,947010262,2099-12-29 00:00:00,5.240782
2903709,947010262,2099-12-30 00:00:00,5.344503
